In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

### Parameters

In [44]:
MASSES = [1, 1, 1, 1]
PARTICLES = len(MASSES)
T = 50
T_PLOT = 100
LEARNING_STEPS = 200

FOURIER_TERMS = 15

W = ( A_c, A_s, B_c, B_s, phi )

In [3]:
def position (W, i, t):
    
    a, b, c, d, phi = W
    
    pos_x, pos_y = 0, 0
    
    for k in range(FOURIER_TERMS + 1):
        
        pos_x += a[k] * np.cos( k * ( t + phi[i] ) ) + b[k] * np.sin( k * ( t + phi[i] ) )
        pos_y += c[k] * np.cos( k * ( t + phi[i] ) ) + d[k] * np.sin( k * ( t + phi[i] ) )
        
    return np.array([pos_x, pos_y])

def speed(W, i, t):
    
    a, b, c, d, phi = W
    
    vel_x, vel_y = 0, 0
    
    for k in range(FOURIER_TERMS + 1):
        
        vel_x += k * ( - a[k] * np.sin( k * ( t + phi[i] ) ) + b[k] * np.cos( k * ( t + phi[i] ) ) )
        vel_y += k * ( - c[k] * np.sin( k * ( t + phi[i] ) ) + d[k] * np.cos( k * ( t + phi[i] ) ) )
        
    return np.array([vel_x, vel_y])
        
def action (W):
    
    S = 0
    
    for t in range(T):
        
        time = 2 * t * np.pi / T
        
        for i in range(PARTICLES):
            
            S += ( MASSES[i] / 2 ) * np.linalg.norm( speed(W, i, time) ) ** 2
            
            for j in range(i + 1, PARTICLES):
                
                S += MASSES[i] * MASSES[j] / ( np.linalg.norm( position(W, i, time) 
                                                             - position(W, j, time) ) )
        
    return S * ( 2 * np.pi / T )
    
def ploteo_trayectorias (W, show=True):
    
    for i in range(PARTICLES):
    
        pos_x, pos_y = [], []
    
        for t in range(T_PLOT):
            
            pos = position(W, i , 2 * t * np.pi / T_PLOT)
            pos_x.append( pos[0] )
            pos_y.append( pos[1] )
            
                        
        plt.scatter(pos[0], pos[1], c = 'black', s = 50 * MASSES[i] )
        
        plt.plot(pos_x, pos_y, "-", c="black", lw = 1)
        
    plt.axis("equal")
    plt.axis('off')
    if show:
        plt.show()
    
    
def grad (W, actualize_bias = False):
    
    a, b, c, d, phi = W

    grad_a = np.zeros( ( FOURIER_TERMS + 1 ) )
    grad_b = np.zeros( ( FOURIER_TERMS + 1 ) )
    grad_c = np.zeros( ( FOURIER_TERMS + 1 ) )
    grad_d = np.zeros( ( FOURIER_TERMS + 1 ) )
    
    for t in range(T):
        
        time = 2 * t * np.pi / T
        
        for i in range( PARTICLES ):
            
            pos_i = position(W, i, time )
            vel_i = speed(W, i, time )
            
            for k in range( FOURIER_TERMS + 1 ):
                
                t_ki = k * ( time + phi[i] )

                if k == 0 and not actualize_bias:    
                    continue
                                
                grad_a [k] += - MASSES[i] * vel_i[0] * np.sin( t_ki ) * k
                grad_b [k] += + MASSES[i] * vel_i[0] * np.cos( t_ki ) * k
                grad_c [k] += - MASSES[i] * vel_i[1] * np.sin( t_ki ) * k
                grad_d [k] += + MASSES[i] * vel_i[1] * np.cos( t_ki ) * k

                for j in range( i + 1, PARTICLES ):
                    
                    t_kj = k * ( time + phi[j] )
                    
                    pos_j = position(W, j, time)
                    vel_j = speed(W, j, time)
                    
                    grad_a [k] += - MASSES[i] * MASSES[j]   \
                                        / ( np.linalg.norm( pos_i - pos_j ) ** 3 ) \
                                        * ( pos_i[0] - pos_j[0] ) \
                                        * ( np.cos( t_ki ) - np.cos( t_kj ) )
                    grad_b [k] += - MASSES[i] * MASSES[j]   \
                                        / ( np.linalg.norm( pos_i - pos_j ) ** 3 ) \
                                        * ( pos_i[0] - pos_j[0] ) \
                                        * ( np.sin( t_ki ) - np.sin( t_kj ) )
                    grad_c [k] += - MASSES[i] * MASSES[j]   \
                                        / ( np.linalg.norm( pos_i - pos_j ) ** 3 ) \
                                        * ( pos_i[1] - pos_j[1] ) \
                                        * ( np.cos( t_ki ) - np.cos( t_kj ) )
                    grad_d [k] += - MASSES[i] * MASSES[j]   \
                                        / ( np.linalg.norm( pos_i - pos_j ) ** 3 ) \
                                        * ( pos_i[1] - pos_j[1] ) \
                                        * ( np.sin( t_ki ) - np.sin( t_kj ) )
                                
                    
    grad_phi = np.zeros( ( PARTICLES ) )

    for t in range(T):
        
        time = 2 * t * np.pi / T
        
        for i in range( PARTICLES ):
        
            sum1, sum2 = 0, 0 
            
            pos_i = position(W, i, time )
            vel_i = speed(W, i, time )

            for k in range( FOURIER_TERMS + 1 ):
                
                t_ki = k * ( time + phi[i] )

                sum1 += k * k * ( a[k] * np.cos( t_ki ) + b[k] * np.sin( t_ki ) )
                sum2 += k * k * ( c[k] * np.cos( t_ki ) + d[k] * np.sin( t_ki ) )

            grad_phi[i] += - MASSES[i] * vel_i[0] * sum1
            grad_phi[i] += - MASSES[i] * vel_i[1] * sum2
            
            for j in range( PARTICLES ):
                
                if j == i:
                    continue
                    
                t_kj = k * ( time + phi[j] )
                    
                pos_j = position(W, j, time )
                vel_j = speed(W, j, time )
                    
                grad_phi[i] += - MASSES[i] * MASSES[j]                       \
                               / ( np.linalg.norm( pos_i - pos_j ) ** 3 )    \
                               * ( ( pos_i[0] - pos_j[0] ) * ( vel_i[0] - vel_j[0] ) +          \
                                   ( pos_i[1] - pos_j[1] ) * ( vel_i[1] - vel_j[0] ) )
                
    
    grad_a = ( (2 * np.pi / T ) ** 2 ) * grad_a
    grad_b = ( (2 * np.pi / T ) ** 2 ) * grad_b
    grad_c = ( (2 * np.pi / T ) ** 2 ) * grad_c
    grad_d = ( (2 * np.pi / T ) ** 2 ) * grad_d
    grad_phi = ( 2 * np.pi / T ) * grad_phi
    
    return np.array([grad_a, grad_b, grad_c, grad_d, grad_phi])

def initialization( PARTICLES, FOURIER_TERMS, mode = "RANDOM", density = 1.0, decay = 0.0 ):
    
      
    a_0 = np.random.normal( size = ( FOURIER_TERMS + 1 ) ) 
    b_0 = np.random.normal( size = ( FOURIER_TERMS + 1 ) ) 
    c_0 = np.random.normal( size = ( FOURIER_TERMS + 1 ) ) 
    d_0 = np.random.normal( size = ( FOURIER_TERMS + 1 ) ) 

    if density < 1.0:
        
        a_0[ np.random.random( a_0.shape ) > density ] = 0
        b_0[ np.random.random( b_0.shape ) > density ] = 0
        c_0[ np.random.random( c_0.shape ) > density ] = 0
        d_0[ np.random.random( d_0.shape ) > density ] = 0

    if decay > 0 :
        
        factor = np.array( [0] + [ k ** ( - decay)  for k in range( 1,  FOURIER_TERMS + 1 ) ] )
        a_0 = a_0 * factor
        b_0 = b_0 * factor
        c_0 = c_0 * factor
        d_0 = d_0 * factor
    
    a_0[0] = 0
    b_0[0] = 0
    c_0[0] = 0
    d_0[0] = 0
    
    phi_0 = np.linspace(0, 2 * np.pi, PARTICLES+1)[0:-1]
        
    return np.array([ a_0, b_0, c_0, d_0, phi_0 ])

## Gradient descent

#### Normal gradient descent

In [4]:
def gradient_descent_0 (LEARNING_RATE, LEARNING_STEPS, grad_stop=1e-4, ploteo=False):
    
    W_0 = initialization( PARTICLES, FOURIER_TERMS, density=0.5, decay = 1)
    Ws = [W_0]
    
    for step in range(LEARNING_STEPS):

        grad_W = grad(Ws[-1])
        
        if np.sum( [ np.linalg.norm(a) for a in grad_W ] ) < grad_stop:
            break
        
        Ws.append( Ws[-1] - LEARNING_RATE * grad_W )
        
        print("Step No", step , " --->", action(Ws[-1]))

    if ploteo == True:
        
        S_improvement = [ action(w) for w in Ws ]
        plt.close()
        plt.semilogy( range(len(S_improvement)), S_improvement )
        plt.xlabel("step")
        plt.ylabel("Action")
        plt.show()
        #plt.savefig( "6-" + str(ii) + "_entrenamiento.png", format="png", dpi=300 )

    return Ws[-1]

#### Gradient descent with Armijo's Rule

In [45]:
def armijo (LEARNING_STEPS, init="RANDOM", W0 = 1, eps=0.2, eta=2):
    
    alpha = 1
    
    if init == "RANDOM":
        W_0 = initialization( PARTICLES, FOURIER_TERMS, density=0.3, decay = 1)
    if init == "GIVEN":
        W_0 = W0
    
    Ws = [W_0]
    
    for step in range(LEARNING_STEPS):

        grad_W = grad(Ws[-1])
                
        delta = 1e-4
        Delta_W = (action(Ws[-1] - delta * grad_W) - action(Ws[-1]) ) / delta
        norm_grad_W = np.abs(Delta_W)         
            
        if step > 5 and np.abs( action(Ws[-1]) - action(Ws[-5]) ) < 1e-10:
            break

        if action(Ws[-1] - alpha * grad_W ) <= action(Ws[-1]) - eps * alpha * norm_grad_W:
            while action(Ws[-1] - alpha * grad_W ) <= action(Ws[-1]) - eps * alpha * norm_grad_W:
                alpha = alpha * eta
            alpha = alpha / eta
        else:
            while action(Ws[-1] - alpha * grad_W ) >  action(Ws[-1]) - eps * alpha * norm_grad_W:
                alpha = alpha / eta
        
        Ws.append( Ws[-1] - alpha * grad_W )
        
        print("Step No", step , " --->", action(Ws[-1]))
        
    return Ws[-1]

In [25]:
def simulated_annealing (LEARNING_STEPS, T0=1):
    
    W = initialization( PARTICLES, FOURIER_TERMS, density=0.3, decay = 1)

    for k in range(LEARNING_STEPS):
        
        temp = np.divide( 100 * (LEARNING_STEPS - k), LEARNING_STEPS )
        
        delta_W = 0.01 * initialization( PARTICLES, FOURIER_TERMS, density=1, decay = 0)
        W_new = W + delta_W
        
        if action(W_new) < action(W):
            W = W_new
        else:
            if np.exp( - ( action(W_new) - action(W) ) / temp ) > np.random.uniform():
                W = W_new
                
        print("Step No", k, " --->", action(W))
                
    return W

In [35]:
def mixture(LEARNING_STEPS_INIT, LEARNING_STEPS_ARMIJO):
    
    W0 = simulated_annealing(LEARNING_STEPS_INIT)
    W  = armijo(LEARNING_STEPS_ARMIJO, init="GIVEN", W0=W0)
    
    return W

In [ ]:
mypath = "/Users/facundosapienza/Dropbox/Compartido/TP Final Optimización/Orbitas - Minimización sin restricciónes"
#mypath = "/home/facu/Dropbox/Compartido/TP Final Optimización/Orbitas - Minimización sin restricciónes"
NUMBER_EXPERIMENTS = 100

for ii in range(NUMBER_EXPERIMENTS):

    print("---------------------")
    
    """
    Normal gradient descent with fixed learning rate
    """
    #W = gradient_descent_0(LEARNING_RATE=1e-2, LEARNING_STEPS, ploteo=True)

    """
    Gradient descent with Armijo's Rule
    """
    #W = armijo(LEARNING_STEPS, eps=0.2)

    """
    Simulated annealing
    """
    #W = simulated_annealing(LEARNING_STEPS=200)
    
    """
    Mixture of models
    """
    W = mixture(LEARNING_STEPS_INIT=100, LEARNING_STEPS_ARMIJO=2000)
    
    plt.close()
    ploteo_trayectorias(W, show=False)
    plt.show()
    
    #plt.savefig( "Coreo_Mixture_2-"+ str(ii) + "_orbitas.png", format="png", dpi=300)

---------------------
Step No 0  ---> 315.0892884009439
Step No 1  ---> 299.7194966856298
Step No 2  ---> 301.66448818340706
Step No 3  ---> 308.332391974791
Step No 4  ---> 297.44261043060675
Step No 5  ---> 301.4441706160689
Step No 6  ---> 318.0354690166997
Step No 7  ---> 324.5964678263633
Step No 8  ---> 324.5964678263633
Step No 9  ---> 329.0939743439022
Step No 10  ---> 319.08273264806377
Step No 11  ---> 332.48062248863835
Step No 12  ---> 338.29607316664493
Step No 13  ---> 341.6038767362109
Step No 14  ---> 340.7906103831296
Step No 15  ---> 339.38796231602663
Step No 16  ---> 339.8620076908915
Step No 17  ---> 355.6148056864229
Step No 18  ---> 372.4399167182418
Step No 19  ---> 406.1568554518342
Step No 20  ---> 406.1568554518342
Step No 21  ---> 406.1568554518342
Step No 22  ---> 406.1568554518342
Step No 23  ---> 406.1568554518342
Step No 24  ---> 406.1568554518342
Step No 25  ---> 406.1568554518342
Step No 26  ---> 406.1568554518342
Step No 27  ---> 406.1568554518342
Ste

Step No 99  ---> 361.1926177811564
Step No 0  ---> 314.16108180216276
Step No 1  ---> 312.0639516678719
Step No 2  ---> 245.43563774188422
Step No 3  ---> 229.7798031046863
Step No 4  ---> 221.1396101762719
Step No 5  ---> 215.486373766465
Step No 6  ---> 214.2499147256585
Step No 7  ---> 211.67442832658833
Step No 8  ---> 199.80669436645454
Step No 9  ---> 190.66424029028815
Step No 10  ---> 182.8147976964895
Step No 11  ---> 181.4781079073422
Step No 12  ---> 179.83397952960001
Step No 13  ---> 177.12887821174866
Step No 14  ---> 166.26526757003418
Step No 15  ---> 155.1890683114819
Step No 16  ---> 148.3160779657175
Step No 17  ---> 147.01928749206587
Step No 18  ---> 144.59900900785982
Step No 19  ---> 142.47135063891594
Step No 20  ---> 138.3766284707722
Step No 21  ---> 132.7258306661737
Step No 22  ---> 130.30668387455432
Step No 23  ---> 128.63298927758777
Step No 24  ---> 124.69106215844529
Step No 25  ---> 122.8231947903272
Step No 26  ---> 120.16740743364001
Step No 27  --->

Step No 225  ---> 110.36291641780325
Step No 226  ---> 110.35717405151598
Step No 227  ---> 110.35250943453356
Step No 228  ---> 110.3474055563489
Step No 229  ---> 110.34139515701551
Step No 230  ---> 110.3376574537971
Step No 231  ---> 110.3352758537292
Step No 232  ---> 110.3293970481207
Step No 233  ---> 110.31905258468255
Step No 234  ---> 110.31437261374361
Step No 235  ---> 110.31019575052645
Step No 236  ---> 110.30366754681316
Step No 237  ---> 110.29861484086915
Step No 238  ---> 110.29500345622341
Step No 239  ---> 110.28798567068446
Step No 240  ---> 110.2842607358343
Step No 241  ---> 110.28147451794867
Step No 242  ---> 110.27547642034615
Step No 243  ---> 110.26269281642466
Step No 244  ---> 110.25948020774905
Step No 245  ---> 110.25492534034396
Step No 246  ---> 110.24934885201128
Step No 247  ---> 110.24320153349503
Step No 248  ---> 110.23911927599498
Step No 249  ---> 110.23665547482344
Step No 250  ---> 110.23387616440576
Step No 251  ---> 110.222747703543
Step No 

Step No 448  ---> 109.52626317310154
Step No 449  ---> 109.52364843849756
Step No 450  ---> 109.52184140347377
Step No 451  ---> 109.51950195910217
Step No 452  ---> 109.5183968863737
Step No 453  ---> 109.51786479099029
Step No 454  ---> 109.51694797924914
Step No 455  ---> 109.51527084556962
Step No 456  ---> 109.51219399777489
Step No 457  ---> 109.51108505012776
Step No 458  ---> 109.50788486723629
Step No 459  ---> 109.50653808272872
Step No 460  ---> 109.50565833736863
Step No 461  ---> 109.50270429300578
Step No 462  ---> 109.49977156341758
Step No 463  ---> 109.49859251387294
Step No 464  ---> 109.49787965366575
Step No 465  ---> 109.4966859282856
Step No 466  ---> 109.49462613629937
Step No 467  ---> 109.49071669704526
Step No 468  ---> 109.48949233124436
Step No 469  ---> 109.48834701994004
Step No 470  ---> 109.48547201185372
Step No 471  ---> 109.48182933510648
Step No 472  ---> 109.48063471018074
Step No 473  ---> 109.47955691594068
Step No 474  ---> 109.47634573052713
Ste

Step No 671  ---> 109.17879798649378
Step No 672  ---> 109.17838839949086
Step No 673  ---> 109.17792132425346
Step No 674  ---> 109.17729077212232
Step No 675  ---> 109.1769009721893
Step No 676  ---> 109.17630694825304
Step No 677  ---> 109.17607241815674
Step No 678  ---> 109.17568573738791
Step No 679  ---> 109.17524242217593
Step No 680  ---> 109.17485139783209
Step No 681  ---> 109.1742243321516
Step No 682  ---> 109.17389722655118
Step No 683  ---> 109.17331608122717
Step No 684  ---> 109.17309869790566
Step No 685  ---> 109.17277865886429
Step No 686  ---> 109.17230135500547
Step No 687  ---> 109.171979486447
Step No 688  ---> 109.17136207113468
Step No 689  ---> 109.17117631453362
Step No 690  ---> 109.17087800717007
Step No 691  ---> 109.17055248691626
Step No 692  ---> 109.16973426437961
Step No 693  ---> 109.16940792752364
Step No 694  ---> 109.16869885457324
Step No 695  ---> 109.16826771494785
Step No 696  ---> 109.1677432309248
Step No 697  ---> 109.16746247935362
Step N

Step No 894  ---> 109.1293730274808
Step No 895  ---> 109.1293022384486
Step No 896  ---> 109.12919943306892
Step No 897  ---> 109.12902454938829
Step No 898  ---> 109.12886897085743
Step No 899  ---> 109.12877458732689
Step No 900  ---> 109.12870001211974
Step No 901  ---> 109.12864858692058
Step No 902  ---> 109.12856677374246
Step No 903  ---> 109.12849832106225
Step No 904  ---> 109.12836644897607
Step No 905  ---> 109.12825468153298
Step No 906  ---> 109.12809453554912
Step No 907  ---> 109.12800240086385
Step No 908  ---> 109.12791347861608
Step No 909  ---> 109.1278341408414
Step No 910  ---> 109.12776972706504
Step No 911  ---> 109.12770929933353
Step No 912  ---> 109.12760088443541
Step No 913  ---> 109.12752348085041
Step No 914  ---> 109.12739556782877
Step No 915  ---> 109.1272913515721
Step No 916  ---> 109.12720715041098
Step No 917  ---> 109.1271528081273
Step No 918  ---> 109.1270720540848
Step No 919  ---> 109.12700494122758
Step No 920  ---> 109.12689199645969
Step No

Step No 1114  ---> 109.09903756309686
Step No 1115  ---> 109.09894329764715
Step No 1116  ---> 109.09886133525069
Step No 1117  ---> 109.09873810679797
Step No 1118  ---> 109.09821778385353
Step No 1119  ---> 109.09701211159974
Step No 1120  ---> 109.09691251345524
Step No 1121  ---> 109.09681595565877
Step No 1122  ---> 109.09672247866102
Step No 1123  ---> 109.09663676650447
Step No 1124  ---> 109.09649629480693
Step No 1125  ---> 109.09599618279876
Step No 1126  ---> 109.0956240503446
Step No 1127  ---> 109.09542371058753
Step No 1128  ---> 109.09508717085626
Step No 1129  ---> 109.09485348440654
Step No 1130  ---> 109.09456736913465
Step No 1131  ---> 109.0943253702848
Step No 1132  ---> 109.09407001896173
Step No 1133  ---> 109.0937912664466
Step No 1134  ---> 109.09356420334244
Step No 1135  ---> 109.09326349683299
Step No 1136  ---> 109.09305782002097
Step No 1137  ---> 109.09274237245499
Step No 1138  ---> 109.09255077916754
Step No 1139  ---> 109.09222731433755
Step No 1140  -

Step No 1331  ---> 109.04991365405547
Step No 1332  ---> 109.04979497140775
Step No 1333  ---> 109.0496786149883
Step No 1334  ---> 109.04951270988249
Step No 1335  ---> 109.04929327609344
Step No 1336  ---> 109.04856042086072
Step No 1337  ---> 109.04820894709513
Step No 1338  ---> 109.04799633833514
Step No 1339  ---> 109.04735130066689
Step No 1340  ---> 109.04722581914663
Step No 1341  ---> 109.04684103264461
Step No 1342  ---> 109.04651070719366
Step No 1343  ---> 109.04630609057304
Step No 1344  ---> 109.0460856730792
Step No 1345  ---> 109.0458953796397
Step No 1346  ---> 109.04581244689079
Step No 1347  ---> 109.04568387464501
Step No 1348  ---> 109.0453880409617
Step No 1349  ---> 109.0451646499762
Step No 1350  ---> 109.04466120065086
Step No 1351  ---> 109.04427734691413
Step No 1352  ---> 109.04410691261376
Step No 1353  ---> 109.04392168965316
Step No 1354  ---> 109.04369232960774
Step No 1355  ---> 109.04343823258888
Step No 1356  ---> 109.04333520865626
Step No 1357  ---

Step No 1548  ---> 109.02728202607037
Step No 1549  ---> 109.0272625005243
Step No 1550  ---> 109.02724546407569
Step No 1551  ---> 109.02722424370653
Step No 1552  ---> 109.02720279512437
Step No 1553  ---> 109.02718367691907
Step No 1554  ---> 109.02716176089697
Step No 1555  ---> 109.02714356473221
Step No 1556  ---> 109.02712696460245
Step No 1557  ---> 109.02710923482032
Step No 1558  ---> 109.02708416320539
Step No 1559  ---> 109.02706828393893
Step No 1560  ---> 109.02704961130284
Step No 1561  ---> 109.02703314610356
Step No 1562  ---> 109.0270142884029
Step No 1563  ---> 109.02699233636434
Step No 1564  ---> 109.02697429283141
Step No 1565  ---> 109.02695397312836
Step No 1566  ---> 109.02693518294433
Step No 1567  ---> 109.02692025134002
Step No 1568  ---> 109.02690294827802
Step No 1569  ---> 109.02687940682148
Step No 1570  ---> 109.0268651922134
Step No 1571  ---> 109.02684798936171
Step No 1572  ---> 109.02683117723451
Step No 1573  ---> 109.02681467080554
Step No 1574  -

Step No 1765  ---> 109.02500364726356
Step No 1766  ---> 109.02500042813622
Step No 1767  ---> 109.02499574302078
Step No 1768  ---> 109.02498963645725
Step No 1769  ---> 109.02498388609612
Step No 1770  ---> 109.02497985835991
Step No 1771  ---> 109.02497685540439
Step No 1772  ---> 109.02497295320393
Step No 1773  ---> 109.02496732415591
Step No 1774  ---> 109.02496124043553
Step No 1775  ---> 109.02495654168393
Step No 1776  ---> 109.02495341254235
Step No 1777  ---> 109.02495015505691
Step No 1778  ---> 109.02494523634037
Step No 1779  ---> 109.0249391420057
Step No 1780  ---> 109.02493379439925
Step No 1781  ---> 109.02493023242283
Step No 1782  ---> 109.02492735325993
Step No 1783  ---> 109.0249232399913
Step No 1784  ---> 109.02491747673336
Step No 1785  ---> 109.02491165521997
Step No 1786  ---> 109.02490745575332
Step No 1787  ---> 109.02490461607584
Step No 1788  ---> 109.02490124343419
Step No 1789  ---> 109.02489609848882
Step No 1790  ---> 109.02489009510619
Step No 1791  